In [14]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import json
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm.notebook import tqdm
from constants import Constants
from kleague_parser import KLeagueParser

base_path = os.path.join(os.path.dirname(os.getcwd()))
print(f"Base path: {base_path}")

Base path: /root/express-v2


In [15]:
%load_ext autoreload
%autoreload 2

pd.set_option('display.width', 250)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

%matplotlib inline
mpl.rcParams['animation.embed_limit'] = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Counting the Number of Matches Satisfying a Given Conditions

In [16]:
team_list = ["울산", "전북", "서울", "대전", "제주", "김천", "김포", "아산", "안산", "광주", "강원", "포항", "대구", "인천", "수원F"]

season = "2024"
match_ids = np.sort([int(f) for f in os.listdir(f"{base_path}/data/bepro/raw") if f[0] != "."])
constants = Constants()

count = 0
for i, match_id in enumerate(match_ids):
    # metadata_path = f"data/{season}/raw_data/{match_id}/{match_id}-match_info.json"
    metadata_path = f"{base_path}/data/bepro/raw/{match_id}/{match_id}_metadata.json"
    with open(metadata_path) as f:
        metadata = json.load(f)
        home_team = constants.TEAM_DICT[metadata["home_team"]["team_name"]]
        away_team = constants.TEAM_DICT[metadata["away_team"]["team_name"]]
        
        if home_team in team_list and away_team in team_list:
            count += 1
            print(f"[{i}] {match_id}: {home_team} vs {away_team}")
        else:
            raise ValueError(f"Invalid team names: {home_team}, {away_team}")

count

[0] 126285: 광주 vs 서울
[1] 126293: 광주 vs 강원
[2] 126298: 포항 vs 광주
[3] 126306: 광주 vs 대구
[4] 126309: 광주 vs 인천
[5] 126315: 김천 vs 광주
[6] 126319: 전북 vs 광주
[7] 126325: 광주 vs 울산
[8] 126332: 광주 vs 수원F
[9] 126341: 제주 vs 광주
[10] 126348: 광주 vs 대전
[11] 126350: 대구 vs 광주
[12] 126356: 광주 vs 전북
[13] 126364: 인천 vs 광주
[14] 126367: 광주 vs 포항
[15] 126378: 서울 vs 광주
[16] 126380: 광주 vs 김천
[17] 126386: 대전 vs 광주
[18] 126391: 수원F vs 광주
[19] 126401: 광주 vs 제주
[20] 126408: 강원 vs 광주
[21] 126411: 울산 vs 광주
[22] 126418: 광주 vs 인천
[23] 126424: 대구 vs 광주
[24] 126429: 광주 vs 수원F
[25] 126433: 전북 vs 광주
[26] 126444: 강원 vs 광주
[27] 126448: 광주 vs 울산
[28] 126455: 대전 vs 광주
[29] 126458: 광주 vs 포항
[30] 126466: 광주 vs 제주
[31] 126473: 김천 vs 광주
[32] 126476: 광주 vs 서울
[33] 153364: 광주 vs 대구
[34] 153373: 인천 vs 광주
[35] 153379: 광주 vs 대전
[36] 153381: 수원F vs 강원
[37] 153385: 제주 vs 광주
[38] 153387: 울산 vs 수원F
[39] 153390: 광주 vs 전북


40

In [39]:
match_id = 126285#match_ids[1]
print(f"Processing match ID: {match_id}")

with open(f"{base_path}/data/bepro/processed/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
    match_dict = pickle.load(f)
    events = match_dict['event_df']
    teams_dict = match_dict['teams']
    metadata = match_dict['meta_data']
    tracking = match_dict['tracking_df']

Processing match ID: 126285


In [40]:
tracking

,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier
0,126285,1.0,0 days 00:00:01.600000,40,alive,4648,12.6493,-19.9514,0.0,0.213345,-0.272586,0.0,0.346149,-0.785569,-0.110482,0.0,0.793300,139210,316,LB,4648,False
1,126285,1.0,0 days 00:00:01.600000,40,alive,4648,4.9128,-3.8996,0.0,-0.809779,-0.095294,0.0,0.815366,0.049218,0.181609,0.0,0.188160,143410,316,CM,4648,False
2,126285,1.0,0 days 00:00:01.600000,40,alive,4648,6.5194,3.6208,0.0,-1.588645,0.919292,0.0,1.835454,-0.771128,0.211322,0.0,0.799559,161110,316,CM,4648,False
3,126285,1.0,0 days 00:00:01.600000,40,alive,4648,-2.1558,9.0551,0.0,-0.692393,-0.584796,0.0,0.906308,0.242714,0.021364,0.0,0.243652,187326,316,CAM,4648,False
4,126285,1.0,0 days 00:00:01.600000,40,alive,4648,-2.1225,18.1590,0.0,-1.126289,0.462982,0.0,1.217736,-0.367498,0.040542,0.0,0.369728,188266,316,RW,4648,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3496732,126285,2.0,0 days 01:37:53.600000,146840,alive,4648,24.7964,-0.4614,0.0,-0.204866,-0.110705,0.0,0.232864,0.843550,0.025362,0.0,0.843931,500140,4648,LB,4648,False
3496733,126285,2.0,0 days 01:37:53.600000,146840,alive,4648,17.5989,8.3096,0.0,0.828691,1.984953,0.0,2.150993,-0.744655,-0.008461,0.0,0.744704,500142,4648,CM,4648,False
3496734,126285,2.0,0 days 01:37:53.600000,146840,alive,4648,26.3027,28.3392,0.0,-1.623980,2.283758,0.0,2.802296,-1.440096,-0.759234,0.0,1.627978,62386,4648,RB,4648,False
3496735,126285,2.0,0 days 01:37:53.600000,146840,alive,4648,7.6053,33.1475,0.0,-0.300878,-0.381421,0.0,0.485808,0.552148,-0.499938,0.0,0.744852,77579,316,CB,4648,False


In [41]:
events

,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,x,y,to_x,to_y,attack_direction
0,Half,1st Half,0,2700000,0,1501,Gwangju FC,20,Kunhee Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.5010,0.5060,0.4276,0.6138,RIGHT
1,Half,1st Half,0,2700000,0,2334,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes Received', 'property':...",0.4276,0.6138,NaN,NaN,RIGHT
2,Half,1st Half,0,2700000,0,3101,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4230,0.6042,0.3712,0.5116,RIGHT
3,Half,1st Half,0,2700000,0,4134,Gwangju FC,14,Hoyeon Jeong,"[{'event_name': 'Passes Received', 'property':...",0.3712,0.5116,NaN,NaN,RIGHT
4,Half,1st Half,0,2700000,0,5334,Gwangju FC,14,Hoyeon Jeong,"[{'event_name': 'Passes', 'property': {'Outcom...",0.3659,0.4809,0.2850,0.4001,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,Half,2nd Half,1,2700000,2700000,5869233,Gwangju FC,18,Yool Heo,"[{'name': 'SPRINT', 'property': {'duration': 5...",0.6262,0.6263,0.5929,0.6315,LEFT
2019,Half,2nd Half,1,2700000,2700000,5869700,Gwangju FC,11,Gabriel,"[{'event_name': 'Passes Received', 'property':...",0.6138,0.9019,NaN,NaN,LEFT
2020,Half,2nd Half,1,2700000,2700000,5869733,Gwangju FC,18,Yool Heo,"[{'name': 'VHIR', 'property': {'duration': 100...",0.5929,0.6315,0.5301,0.6411,LEFT
2021,Half,2nd Half,1,2700000,2700000,5873433,Gwangju FC,11,Gabriel,"[{'event_name': 'Take-on', 'property': {'Outco...",0.6339,1.0000,NaN,NaN,LEFT


In [42]:
events

,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,player_shirt_number,player_name,events,x,y,to_x,to_y,attack_direction
0,Half,1st Half,0,2700000,0,1501,Gwangju FC,20,Kunhee Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.5010,0.5060,0.4276,0.6138,RIGHT
1,Half,1st Half,0,2700000,0,2334,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes Received', 'property':...",0.4276,0.6138,NaN,NaN,RIGHT
2,Half,1st Half,0,2700000,0,3101,Gwangju FC,10,Huigyun Lee,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4230,0.6042,0.3712,0.5116,RIGHT
3,Half,1st Half,0,2700000,0,4134,Gwangju FC,14,Hoyeon Jeong,"[{'event_name': 'Passes Received', 'property':...",0.3712,0.5116,NaN,NaN,RIGHT
4,Half,1st Half,0,2700000,0,5334,Gwangju FC,14,Hoyeon Jeong,"[{'event_name': 'Passes', 'property': {'Outcom...",0.3659,0.4809,0.2850,0.4001,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,Half,2nd Half,1,2700000,2700000,5869233,Gwangju FC,18,Yool Heo,"[{'name': 'SPRINT', 'property': {'duration': 5...",0.6262,0.6263,0.5929,0.6315,LEFT
2019,Half,2nd Half,1,2700000,2700000,5869700,Gwangju FC,11,Gabriel,"[{'event_name': 'Passes Received', 'property':...",0.6138,0.9019,NaN,NaN,LEFT
2020,Half,2nd Half,1,2700000,2700000,5869733,Gwangju FC,18,Yool Heo,"[{'name': 'VHIR', 'property': {'duration': 100...",0.5929,0.6315,0.5301,0.6411,LEFT
2021,Half,2nd Half,1,2700000,2700000,5873433,Gwangju FC,11,Gabriel,"[{'event_name': 'Take-on', 'property': {'Outco...",0.6339,1.0000,NaN,NaN,LEFT


### Parsing Match Data

In [35]:
total_mismatches = []
for match_id in tqdm(match_ids):

    with open(f"{base_path}/data/bepro/processed/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
        match_dict = pickle.load(f)
        events = match_dict['event_df']
        teams_dict = match_dict['teams']
        metadata = match_dict['meta_data']
        tracking = match_dict['tracking_df']

    parser = KLeagueParser(match_id, data_dir=f"{base_path}/data/bepro/processed")
    parser.parse_events()
    parser.parse_ball_xy()

    parser.combine_events_and_ball_xy()
    parser.parse_player_xy()
    parser.combine_player_and_ball_xy()

    mismatch = parser.mislabels

    if mismatch is None:
        print(f"No mismatches found for match ID: {match_id}")
        continue
    mismatch["match_id"] = match_id
    total_mismatches.append(mismatch)

total_mismatches_df = pd.concat(total_mismatches, ignore_index=True)
total_mismatches_df

  0%|          | 0/40 [00:00<?, ?it/s]

,session,time,annotated,corrected,ball_error,match_id
0,1,53.160,H30,H30,6.028407,126285
1,1,54.760,H28,H28,7.133459,126285
2,1,56.360,H28,H28,6.372795,126285
3,1,57.520,A30,A21,6.601461,126285
4,1,119.000,A21,A21,6.229558,126285
...,...,...,...,...,...,...
1429,1,2643.813,A39,A30,14.712212,153390
1430,1,2800.173,H14,H55,15.601394,153390
1431,2,1486.240,A27,A09,24.160832,153390
1432,2,1493.800,H55,H40,15.619778,153390


In [38]:
(1434-570) /40

21.6

In [36]:
total_mismatches_df.groupby("match_id").size()

match_id
126285    570
126293     27
126298     27
126306     19
126309      8
126315     25
126319     40
126325     10
126332     17
126341     17
126348     25
126350     29
126356     13
126364     11
126367     27
126378     18
126380     13
126386     14
126391      8
126401     19
126408     19
126411     27
126418     27
126424     19
126429     21
126433     23
126444     38
126448     20
126455     23
126458     29
126466     27
126473     17
126476      6
153364     75
153373     19
153379     24
153381     26
153385     17
153387     23
153390     17
dtype: int64

In [43]:
parser = KLeagueParser(match_id, data_dir=f"{base_path}/data/bepro/processed")
parser.parse_events()
parser.parse_ball_xy()

parser.combine_events_and_ball_xy()
parser.events

,event_id,session,time,home_away,player_code,event_types,x,y
0,0,1,1.501,H,H20,"[{'event_name': 'Passes', 'property': {'Outcom...",-0.817000,1.287600
1,1,1,2.334,H,H10,"[{'event_name': 'Passes Received', 'property':...",-7.523614,7.666288
2,2,1,3.101,H,H10,"[{'event_name': 'Passes', 'property': {'Outcom...",-8.176710,6.976466
3,3,1,4.134,H,H14,"[{'event_name': 'Passes Received', 'property':...",-13.471912,0.855635
4,4,1,5.334,H,H14,"[{'event_name': 'Passes', 'property': {'Outcom...",-14.068108,-1.271192
...,...,...,...,...,...,...,...,...
3355,3905,2,3165.100,H,H27,"[{'event_name': 'Clearances', 'property': {}}]",42.263734,-2.462098
3356,3906,2,3165.833,H,H55,"[{'event_name': 'Recoveries', 'property': {}},...",34.365209,2.245112
3357,3907,2,3167.033,H,H55,"[{'event_name': 'Passes', 'property': {'Outcom...",35.804018,5.361247
3364,3914,2,3169.700,H,H11,"[{'event_name': 'Passes Received', 'property':...",12.091822,27.188117


In [44]:
parser.players

,team_id,player_id,team_name,squad_num,player_name,home_away,player_code,id,t0,t1,s0,s1
0,4648,250079,광주,1,Kyeongmin Kim,H,H01,250079,1.6,3173.60,1.0,2.0
1,4648,500133,광주,4,Alexandar Popovic,H,H04,500133,1.6,3173.60,1.0,2.0
2,4648,62365,광주,6,Youngkyu Ahn,H,H06,62365,1.6,2203.72,1.0,2.0
3,4648,188178,광주,7,Jisung Eom,H,H07,188178,1366.4,3173.60,1.0,2.0
4,4648,77414,광주,10,Huigyun Lee,H,H10,77414,1.6,1170.28,1.0,2.0
5,4648,500135,광주,11,Gabriel,H,H11,500135,1.6,3173.60,1.0,2.0
6,4648,62386,광주,13,Hyeonseok Doo,H,H13,62386,1.6,3173.60,1.0,2.0
7,4648,250101,광주,14,Hoyeon Jeong,H,H14,250101,1.6,3173.60,1.0,2.0
8,4648,500138,광주,15,Junsu Byeon,H,H15,500138,2204.8,3173.60,2.0,2.0
9,4648,145703,광주,18,Yool Heo,H,H18,145703,2210.6,3173.60,2.0,2.0


In [45]:
parser.events

,event_id,session,time,home_away,player_code,event_types,x,y
0,0,1,1.501,H,H20,"[{'event_name': 'Passes', 'property': {'Outcom...",-0.817000,1.287600
1,1,1,2.334,H,H10,"[{'event_name': 'Passes Received', 'property':...",-7.523614,7.666288
2,2,1,3.101,H,H10,"[{'event_name': 'Passes', 'property': {'Outcom...",-8.176710,6.976466
3,3,1,4.134,H,H14,"[{'event_name': 'Passes Received', 'property':...",-13.471912,0.855635
4,4,1,5.334,H,H14,"[{'event_name': 'Passes', 'property': {'Outcom...",-14.068108,-1.271192
...,...,...,...,...,...,...,...,...
3355,3905,2,3165.100,H,H27,"[{'event_name': 'Clearances', 'property': {}}]",42.263734,-2.462098
3356,3906,2,3165.833,H,H55,"[{'event_name': 'Recoveries', 'property': {}},...",34.365209,2.245112
3357,3907,2,3167.033,H,H55,"[{'event_name': 'Passes', 'property': {'Outcom...",35.804018,5.361247
3364,3914,2,3169.700,H,H11,"[{'event_name': 'Passes Received', 'property':...",12.091822,27.188117


In [46]:
parser.parse_player_xy()
if parser.player_xy is not None:
    parser.combine_player_and_ball_xy()
    # parser.split_into_episodes()
    # parser.calc_running_features()
    # parser.save()
    
parser.traces

,frame,event_id,session,phase,player_code,event_types,ball_x,ball_y,time,A03_x,A03_y,A06_x,A06_y,A07_x,A07_y,...,H15_y,H18_x,H18_y,H20_x,H20_y,H27_x,H27_y,H28_x,H28_y,H30_x,H30_y,H55_x,H55_y,H77_x,H77_y
0,0,0.0,1,1,H20,"[{'event_name': 'Passes', 'property': {'Outcom...",-0.817000,1.2876,1.60,14.548100,4.612700,6.519400,3.620800,NaN,NaN,...,NaN,NaN,NaN,-3.396100,0.460700,-20.596300,17.767500,-3.612700,20.775700,-9.408800,-9.819000,NaN,NaN,NaN,NaN
1,1,NaN,1,1,NaN,NaN,NaN,NaN,1.64,14.489500,4.619406,6.454009,3.658546,NaN,NaN,...,NaN,NaN,NaN,-3.392839,0.467131,-20.596300,17.804684,-3.609632,20.784107,-9.407551,-9.818958,NaN,NaN,NaN,NaN
2,2,NaN,1,1,NaN,NaN,NaN,NaN,1.68,14.429711,4.626499,6.387516,3.696035,NaN,NaN,...,NaN,NaN,NaN,-3.388278,0.472151,-20.596300,17.842351,-3.606134,20.794344,-9.405882,-9.817430,NaN,NaN,NaN,NaN
3,3,NaN,1,1,NaN,NaN,NaN,NaN,1.72,14.368414,4.634228,6.319382,3.733161,NaN,NaN,...,NaN,NaN,NaN,-3.382201,0.475823,-20.596300,17.880508,-3.602015,20.806526,-9.403760,-9.813302,NaN,NaN,NaN,NaN
4,4,NaN,1,1,NaN,NaN,NaN,NaN,1.76,14.304021,4.643164,6.247565,3.771084,NaN,NaN,...,NaN,NaN,NaN,-3.374162,0.478688,-20.599176,17.920163,-3.597172,20.820980,-9.400654,-9.808060,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152038,152038,3916.0,2,8,H11,"[{'event_name': 'Take-on', 'property': {'Outco...",14.134579,34.0000,3173.44,-7.819239,9.030708,12.587937,20.039741,10.647920,33.140211,...,16.733195,-6.927077,11.411870,NaN,NaN,24.837292,-0.443267,NaN,NaN,NaN,NaN,17.459054,7.991933,-2.757753,-9.674478
152039,152039,NaN,2,8,NaN,NaN,NaN,NaN,3173.48,-7.862749,9.082466,12.533095,20.178569,10.509245,33.172561,...,16.790950,-6.978432,11.437919,NaN,NaN,24.825072,-0.448157,NaN,NaN,NaN,NaN,17.495918,8.071148,-2.809724,-9.729848
152040,152040,NaN,2,8,NaN,NaN,NaN,NaN,3173.52,-7.905000,9.132756,12.476331,20.313329,10.379547,33.203852,...,16.847403,-7.026141,11.462881,NaN,NaN,24.814227,-0.452949,NaN,NaN,NaN,NaN,17.531312,8.149995,-2.860019,-9.781088
152041,152041,NaN,2,8,NaN,NaN,NaN,NaN,3173.56,-7.947134,9.183016,12.416020,20.447411,10.256091,33.235654,...,16.903924,-7.071623,11.488281,NaN,NaN,24.804516,-0.457476,NaN,NaN,NaN,NaN,17.566236,8.230794,-2.909997,-9.830237


In [53]:
parser.mislabels

,,session,time,annotated,corrected,ball_error
frame,event_id,,,,,
1289,53.0,1,53.16,H30,H30,6.028407
1329,57.0,1,54.76,H28,H28,7.133459
1369,59.0,1,56.36,H28,H28,6.372795
1398,61.0,1,57.52,A30,A21,6.601461
2935,64.0,1,119.00,A21,A21,6.229558
...,...,...,...,...,...,...
151794,3903.0,2,3163.68,A88,A88,5.496444
151830,3905.0,2,3165.12,H27,H01,6.510660
151848,3906.0,2,3165.84,H55,H04,5.357652


In [ ]:
parser.mislabels[parser.mislabels["annotated"] != parser.mislabels["corrected"]]

,,session,time,annotated,corrected,ball_error
frame,event_id,,,,,
1398,61.0,1,57.52,A30,A21,6.601461
5431,170.0,1,218.84,A03,A21,8.077532
6007,171.0,1,241.88,H13,H30,6.694480
8853,245.0,1,355.72,H30,H20,5.805716
9467,259.0,1,380.28,A06,A21,6.953342
...,...,...,...,...,...,...
145811,3821.0,2,2924.36,H07,H27,6.353429
151661,3896.0,2,3158.36,H18,H55,6.145174
151830,3905.0,2,3165.12,H27,H01,6.510660


In [49]:
team_sheets = pd.concat([teams_dict["Home"], teams_dict["Away"]], ignore_index=True)
player_code_to_player_id = {f"{row['team'][0]}{int(row['jID']):02d}": row['pID'] for _, row in team_sheets.iterrows()}
player_code_to_player_id

{'H01': '250079',
 'H04': '500133',
 'H06': '62365',
 'H13': '62386',
 'H27': '500140',
 'H11': '500135',
 'H14': '250101',
 'H28': '500141',
 'H30': '500139',
 'H10': '77414',
 'H20': '250102',
 'H21': '250099',
 'H05': '250108',
 'H15': '500138',
 'H07': '188178',
 'H24': '360389',
 'H47': '360166',
 'H55': '500142',
 'H18': '145703',
 'H77': '369783',
 'A21': '344466',
 'A17': '139210',
 'A30': '77579',
 'A03': '344467',
 'A50': '62009',
 'A06': '161110',
 'A66': '143410',
 'A26': '187326',
 'A32': '62038',
 'A09': '248890',
 'A11': '188266',
 'A01': '77582',
 'A88': '188267',
 'A14': '500125',
 'A29': '500129',
 'A25': '188258',
 'A07': '344465',
 'A10': '500124',
 'A19': '344552',
 'A90': '286696'}

In [50]:
parse_events = parser.events
parse_events["player_id"] = parse_events["player_code"].map(player_code_to_player_id)
parse_events = parse_events[["event_id", "player_id"]]
parse_events

,event_id,player_id
0,0,250102
1,1,77414
2,2,77414
3,3,250101
4,4,250101
...,...,...
1842,3905,250079
1843,3906,500133
1844,3907,500138
1845,3914,500135


In [51]:
raw_events = events.reset_index(drop=True)
raw_events["event_id"] = range(len(events))
# player정보는 재매핑
raw_events = raw_events[['event_id', 'period_type', 'period_name', 'period_order', 
                         'period_duration', 'period_start_time', 'event_time', 
                         'team_name', 'events', 'x', 'y', 'to_x', 'to_y', 'attack_direction']]
raw_events

,event_id,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,events,x,y,to_x,to_y,attack_direction
0,0,Half,1st Half,0,2700000,0,1501,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.5010,0.5060,0.4276,0.6138,RIGHT
1,1,Half,1st Half,0,2700000,0,2334,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.4276,0.6138,NaN,NaN,RIGHT
2,2,Half,1st Half,0,2700000,0,3101,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4230,0.6042,0.3712,0.5116,RIGHT
3,3,Half,1st Half,0,2700000,0,4134,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.3712,0.5116,NaN,NaN,RIGHT
4,4,Half,1st Half,0,2700000,0,5334,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.3659,0.4809,0.2850,0.4001,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3913,3913,Half,2nd Half,1,2700000,2700000,5869233,Gwangju FC,"[{'name': 'SPRINT', 'property': {'duration': 5...",0.6262,0.6263,0.5929,0.6315,LEFT
3914,3914,Half,2nd Half,1,2700000,2700000,5869700,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.6138,0.9019,NaN,NaN,LEFT
3915,3915,Half,2nd Half,1,2700000,2700000,5869733,Gwangju FC,"[{'name': 'VHIR', 'property': {'duration': 100...",0.5929,0.6315,0.5301,0.6411,LEFT
3916,3916,Half,2nd Half,1,2700000,2700000,5873433,Gwangju FC,"[{'event_name': 'Take-on', 'property': {'Outco...",0.6339,1.0000,NaN,NaN,LEFT


In [52]:
raw_events.merge(parse_events, on="event_id", how="left")

,event_id,period_type,period_name,period_order,period_duration,period_start_time,event_time,team_name,events,x,y,to_x,to_y,attack_direction,player_id
0,0,Half,1st Half,0,2700000,0,1501,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.5010,0.5060,0.4276,0.6138,RIGHT,250102
1,1,Half,1st Half,0,2700000,0,2334,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.4276,0.6138,NaN,NaN,RIGHT,77414
2,2,Half,1st Half,0,2700000,0,3101,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.4230,0.6042,0.3712,0.5116,RIGHT,77414
3,3,Half,1st Half,0,2700000,0,4134,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.3712,0.5116,NaN,NaN,RIGHT,250101
4,4,Half,1st Half,0,2700000,0,5334,Gwangju FC,"[{'event_name': 'Passes', 'property': {'Outcom...",0.3659,0.4809,0.2850,0.4001,RIGHT,250101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3913,3913,Half,2nd Half,1,2700000,2700000,5869233,Gwangju FC,"[{'name': 'SPRINT', 'property': {'duration': 5...",0.6262,0.6263,0.5929,0.6315,LEFT,NaN
3914,3914,Half,2nd Half,1,2700000,2700000,5869700,Gwangju FC,"[{'event_name': 'Passes Received', 'property':...",0.6138,0.9019,NaN,NaN,LEFT,500135
3915,3915,Half,2nd Half,1,2700000,2700000,5869733,Gwangju FC,"[{'name': 'VHIR', 'property': {'duration': 100...",0.5929,0.6315,0.5301,0.6411,LEFT,NaN
3916,3916,Half,2nd Half,1,2700000,2700000,5873433,Gwangju FC,"[{'event_name': 'Take-on', 'property': {'Outco...",0.6339,1.0000,NaN,NaN,LEFT,500135
